# Active Learning for MLFF. 


This is a notebook to test active learning workflow for different kinds of MLFF. 

## Import Libraries

In [48]:
from ase.io import read,write
import os
import argparse
from chgnet.model.dynamics import CHGNetCalculator
from chgnet.model import CHGNet
import contextlib
import numpy as np 
import matplotlib.pyplot as plt
import sys
sys.path.append('/mnt/e/Documents/GitHub/MLFF_training/Training')
import ase_active_learning as al 
from ase.visualize import view
import ipywidgets as widgets
from IPython.display import display
from IPython.display import display, clear_output
from ase.visualize import view
import matplotlib.pyplot as plt
import importlib

In [70]:
importlib.reload(al)

<module 'ase_active_learning' from '/mnt/e/Documents/GitHub/MLFF_training/Training/ase_active_learning.py'>

## Get configuration space

From a file that is able to be read by ASE , we are able to import structures. 

In [35]:
configurations = al.get_configuration_space('/home/nacho/doctorate/LLZO/configuration_space',10)
len(configurations)

400

In [9]:
view(configurations[399], viewer='x3d')

## Load MLFF models

Loads all models from a the target directory so that we can test their deviation. Preferably choose more than 3. 

We are loading  models of our reduced data. We are going to use CPU for this test. 

In [26]:
device = 'cpu'
models = al.load_models('/home/nacho/doctorate/LLZO/model/reduced_data/',device=device)
len(models)

3

We set the calculator for the model

In [36]:
configurations = al.set_calculator(configurations,'CHGnet',models,device=device)

CHGNet will run on cpu
CHGNet will run on cpu
CHGNet will run on cpu


## Obtain the standard deviation after calculator energy,forces,and stress using our 3 MLFF models. 

This step will take the longest as it's calculating the energies of the configuration space X how many models you have. This step is incredibly sped-up with GPU's. 

In [ ]:
energies,std_dev,atoms_lists = al.calculate_energies_and_std(configurations)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(std_dev, bins=20, edgecolor='black', alpha=0.7)
plt.title('Distribution of Standard Deviations')
plt.xlabel('Standard Deviation')
plt.ylabel('Frequency')
plt.axvline(x=np.percentile(std_dev, 98), color='r', linestyle='--', label='98th Percentile')
plt.legend()

In [ ]:
energy_short = energies[:400]
plt.scatter(energy_short,std_dev)